# Connectome Pipeline

Hi and welcome to the Connectome Pipeline!

## 1. Preprocessing

In the first step, you will preprocess the CONN Matlab files to an analysis ready dataset.

Here is an overview on the parameters for the preprocessing pipeline. Parameters marked with a (*) are optional.


+    *matlab_dir*: path to matlab files
+    *excel_path*: path to excel list
+    *preprocessing_type*: conn for connectivity matrix or "aggregation" for aggregated conn matrix
+    *export_file**: If false return as pd dataframe
+    *write_dir**: path where to write the dataset to if save_file = True
+    *network**: Yeo7 or Yeo17 network (only applicable if preprocessing_type = aggregation)
+    *statistic**: Summary statistic to be applied (only applicable if preprocessing_type = aggregation)
+    *upper**: boolean whether only upper diagonal elements of connecivity matrices should be used
+    *file_format**: Pass "h5" for further modelling in python or "csv" for R (default "csv")

In [1]:
import os
import pandas as pd

In [4]:
from connectome.preprocessing.preprocessing_matlab_files import preprocess_mat_files

In [12]:
matlab_dir = r"C:\Users\katha\Downloads\Test\raw_246\conn_data/" # Enter the directory for the matlab files
excel_path = r"C:\Users\katha\Downloads\Test/raw_246/example_data.xlsx" # Enter the directory for the corresponding excel sheet
preprocessing_type = 'aggregation'
write_dir = "" # ...
export_file = False # rename to export file

In [15]:
df = preprocess_mat_files(matlab_dir = matlab_dir, excel_path = excel_path, preprocessing_type = preprocessing_type,
                          write_dir = write_dir, export_file = export_file)

loading files


OSError: Unable to open file (file signature not found)

In [5]:
df.head()

,ConnID,Repseudonym,siteid,age,visdat,sex,prmdiag,edyears,MEM_score,Apoe,...,242_243,242_244,242_245,242_246,243_244,243_245,243_246,244_245,244_246,245_246
0,1,0a8d02f2b,11,66,17.08.2016,0,2,17,0.054016,0,...,0.655094,0.547549,-0.256346,0.0799051,0.504077,0.0314353,0.180466,0.0953076,-0.062235,0.210861
1,2,0a71a953d,17,72,30.03.2015,0,1,20,-0.468749,1,...,0.795011,0.624784,-0.115689,0.0555967,0.684795,0.212303,0.193802,-0.117644,-0.0928372,0.621967
2,3,0a61339db,11,72,13.05.2015,1,1,16,-0.093521,0,...,0.209853,0.166607,0.0561474,0.159351,0.721864,0.107519,0.329301,0.0552936,0.39387,0.269579
3,4,0b28aed58,17,76,18.01.2016,0,1,20,0.466027,0,...,0.547318,0.724798,0.205874,0.193587,0.855264,0.0857812,0.333715,0.231567,0.28501,0.780887
4,5,0c1c5ae77,8,64,12.03.2015,1,1,13,1.4521,0,...,0.901057,0.547402,-0.0375842,0.0894263,0.509276,0.122451,0.308097,0.330322,0.360479,0.404907


## 2. Modelling

In the second step, you can decide between running the new input files on a pretrained model or train a new model

### 2.1  Data preparation
Preparation of the data for modelling. Creates the target variable, drops unnecessary columns, performs a train/test split (if wanted). \\
The user has to specify:
- *classification*: is it a classification task (True) or a regression task (False)
- *columns_drop*: which variables shoulnd't be used for modelling
- *target*: what is the name of the target variable
- *y_0, y_1* (only relevant for classification task): which values of the target variable are 0, which are 1
- *train_size*: size of the training data
- *seed*: a seed to ensure reproducibility of train/test split
- split: should a train/test split be performed or not? 

In [6]:
from connectome.preprocessing.data_preparation import prepare_data

In [7]:
classification = True
columns_drop = ["ConnID", "Repseudonym", "siteid", "visdat", "MEM_score", "Apoe", "IDs"]
target = "prmdiag"
y_0 = [0]
y_1 = [2, 3]
train_size = 0.8
seed = 1855
split = True

In [8]:
# preparation of data
X_train, y_train, X_test, y_test = prepare_data(data = df, classification = classification,
                                                columns_drop = columns_drop, target = target, y_0 = y_0, y_1 = y_1,
                                                train_size = train_size, seed = seed, split = split)



### 2.2 Run Model or get pretrained model

Selection which model should be used and whether a pretrained model or newly trained model is desired.

You can find a selection fo pretrained models under the models folder.

The user has to specify:
- X_train: training data coming from the previous step
- y_train: values of target variable for the training data coming from the previous step
- model: which model should be used (options are: "elnet" for elastic net, "gboost" for gradient boosting, "rf" for random forest and "cnn" for convolutional neural network)
- pretrained: is a pretrained model wanted or should the training data be used to fit a new one. (True = pretrained, False = new fit)
- model_path: the full path to the desired pretrained model if one should be used

In [9]:
from connectome.models.framework import model_framework

In [10]:
model = model_framework(X_train = X_train,
                        y_train = y_train,
                        model = "gboost",
                        pretrained = False,
                        model_path = None)

## 3. Model Evaluation

In this step you can now evaluate the Model on a set of prespecified metrics.

+ For Classification: Accuracy, Precision, Recall, F1 and AUC
+ For Regression: MSE, MAE and R2

Checkout https://scikit-learn.org/stable/modules/classes.html#module-sklearn.metrics for details

In [11]:
from connectome.models.evaluation import model_evaluation
from connectome.models.brainnet_cnn import preprocess_test_data_for_cnn

In [137]:
# If a cnn Model was trained, transform the test_dataset to the right input format for the CNN with the same settings
if model.name == 'BrainNetCNN':
    X_test, y_test = preprocess_test_data_for_cnn(X_test, y_test, aggregation=False, reorder=False)

Turning flat array to matrix


In [165]:
model_evaluation(model, X_test, y_test)

,Accuracy,Precision,Recall,F1,AUC
0,0.680851,0.698413,0.8,0.745763,0.677855


## 4. Feature Visualization and Interpretation

In the final step you can choose between several feature visualization and interpretation techniques.

The user has to specify:
+        model: the model from  step 2
+        X: X_test dataframe
+        y: Target test dataframe
+        viz_method: Choice  of "GFI" , "GFI_only", "FI" , "FI_only", "elastic_net", "shapley" and "feature_attribution"

Visualization methods:
+ GFI: Grouped Permutation Feature Importance (based on yeo7 network)
+ GFI_only: Group only Permutation Feature Importance (based on yeo7 network)
+ FI: Permutation Feature Importance
+ FI_only: Version of Group only Permutation Feature Importance but for every feature, not groups
+ elastic_net: Visualization of the elastic net coefficients
+ shapley: Summary plot for shapley values
+ feature_attribution: Neural Network Visulization with Saliency Maps

For more details and customization of plots see our documentation.

In [139]:
from connectome.visualization.viz_framework import visualization_framework

In [175]:
viz = visualization_framework(model=model,X=X_test,
                              y=y_test, viz_method="feature_attribution", method='saliency', average=True, ordered = True)

In [ ]:
viz[0]